In [34]:
!git clone https://github.com/gabrielagcam/Detec-o-precoce-de-AVCs.git

fatal: destination path 'Detec-o-precoce-de-AVCs' already exists and is not an empty directory.


In [35]:
# Instalação necessária
!pip install tensorflow_decision_forests

import pandas as pd
import tensorflow_decision_forests as tfdf
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [36]:
# Carregar os dados tratados
df = pd.read_csv("Detec-o-precoce-de-AVCs/Data/Stroke.csv")

# Separar X e y
X = df.drop('stroke', axis=1)
y = df['stroke']

In [37]:
# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [38]:
# Reunir novamente para uso no TFDF
train_df = X_train.copy()
train_df['stroke'] = y_train

test_df = X_test.copy()
test_df['stroke'] = y_test

# Converter para dataset do TFDF
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_df, label="stroke")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_df, label="stroke")

In [39]:
# Selecionando melhor número de árvores

best_score = 0
best_k = 0

for num_trees in range(10, 301, 10):
    print(f"Treinando com num_trees={num_trees}...")

    model = tfdf.keras.RandomForestModel(num_trees=num_trees, random_seed=42)
    model.fit(train_ds)

    # Compile the model with the desired metric
    model.compile(metrics=["accuracy"])

    evaluation = model.evaluate(test_ds, return_dict=True)  # Remove metrics argument
    accuracy = evaluation["accuracy"]  # Pegando a métrica de acurácia

    print(f"Acurácia para num_trees={num_trees}: {accuracy:.4f}")

    if accuracy > best_score:
        best_score = accuracy
        best_k = num_trees

print(f"\nMelhor num_trees encontrado: {best_k} com acurácia {best_score:.4f}")

Treinando com num_trees=10...
Use /tmp/tmp4kpqisz_ as temporary training directory
Reading training dataset...
Training dataset read in 0:00:00.904273. Found 7572 examples.
Training model...
Model trained in 0:00:00.425359
Compiling model...
Model compiled.
2/2 [==============================] - 1s 106ms/step - loss: 0.0000e+00 - accuracy: 0.9636
Acurácia para num_trees=10: 0.9636
Treinando com num_trees=20...
Use /tmp/tmputn7bh5q as temporary training directory
Reading training dataset...
Training dataset read in 0:00:05.066428. Found 7572 examples.
Training model...
Model trained in 0:00:01.531627
Compiling model...
Model compiled.
2/2 [==============================] - 1s 180ms/step - loss: 0.0000e+00 - accuracy: 0.9530
Acurácia para num_trees=20: 0.9530
Treinando com num_trees=30...
Use /tmp/tmp2r96d_8x as temporary training directory
Reading training dataset...
Training dataset read in 0:00:02.513178. Found 7572 examples.
Training model...
Model trained in 0:00:02.321190
Compiling

In [41]:
# Avaliação
evaluation = model.evaluate(test_ds, return_dict=True)
print(f"Avaliação no conjunto de teste: {evaluation}")

2/2 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 0.9599
Avaliação no conjunto de teste: {'loss': 0.0, 'accuracy': 0.9598732590675354}


In [42]:
# Predição
y_pred_proba = model.predict(test_ds)
y_pred = (y_pred_proba > 0.5).astype(int).reshape(-1)

2/2 [==============================] - 0s 37ms/step


In [43]:
# Relatórios
print("\nRelatório de Classificação:\n", classification_report(y_test, y_pred))
print("Matriz de Confusão:\n", confusion_matrix(y_test, y_pred))


Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96       951
           1       0.93      1.00      0.96       943

    accuracy                           0.96      1894
   macro avg       0.96      0.96      0.96      1894
weighted avg       0.96      0.96      0.96      1894

Matriz de Confusão:
 [[875  76]
 [  0 943]]
